In [ ]:
!pip install efficientnet_pytorch

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

import torch
import torch.nn as nn
import torch.optim as optim

from torchvision.transforms import transforms
from torch.utils.data import DataLoader
from torchvision import datasets

from efficientnet_pytorch import EfficientNet

import os
import cv2
import random
from PIL import Image
from tqdm import tqdm
import warnings
warnings.filterwarnings("ignore")

In [ ]:
BASE_PATH = '/kaggle/input/soil-classification-image-data/Soil_Dataset/'

In [ ]:
def seed_everything(seed=42):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True
    
seed_everything(42)

In [ ]:
from random import uniform
from torchvision.transforms import functional as F

In [ ]:
class RandomBrightness(object):
    def __init__(self,rng=(0.75,1.25)):
        self.rng = rng
    def __call__(self,img):
        rand = uniform(self.rng[0],
                         self.rng[1])
        return F.adjust_brightness(img,rand)
    
class RandomContrast(object):
    def __init__(self,rng=(0.75,1.25)):
        self.rng = rng
    def __call__(self,img):
        rand = uniform(self.rng[0],
                         self.rng[1])
        return F.adjust_contrast(img,rand)

class RandomGamma(object):
    def __init__(self,rng=(0.75,1.25)):
        self.rng = rng
    def __call__(self,img):
        rand = uniform(self.rng[0],
                         self.rng[1])
        return F.adjust_gamma(img,rand)
    
class RandomHue(object):
    def __init__(self,rng=(-0.15,0.15)):
        self.rng = rng
    def __call__(self,img):
        rand = uniform(self.rng[0],
                         self.rng[1])
        return F.adjust_hue(img,rand)
    
class RandomSat(object):
    def __init__(self,rng=(0.75,1.25)):
        self.rng = rng
    def __call__(self,img):
        rand = uniform(self.rng[0],
                         self.rng[1])
        return F.adjust_saturation(img,rand)

augment = transforms.Compose([
            transforms.RandomApply([
                transforms.RandomRotation(10),
                transforms.RandomHorizontalFlip(),
                transforms.RandomVerticalFlip(),
                RandomBrightness(),
                RandomContrast(),
                RandomGamma(),
                RandomSat(),
                transforms.RandomPerspective(),
                transforms.RandomAffine(degrees=359,translate=(0.2,0.2),shear=(20,20,20,20))],0.8)
            ])

transform = transforms.Compose([
            transforms.Resize((224,224)),
            transforms.ToTensor(),
            transforms.Normalize([0.485, 0.456, 0.406],
                                 [0.229, 0.224, 0.225])
            ])

transform_test = transforms.Compose([
            transforms.ToPILImage(),
            transforms.Resize((224,224)),
            transforms.ToTensor(),
            transforms.Normalize([0.485, 0.456, 0.406],
                                 [0.229, 0.224, 0.225])
            ])

In [ ]:
!mkdir data/
!mkdir data/Alluvial_Soil
!mkdir data/Black_Soil
!mkdir data/Clay_Soil
!mkdir data/Red_Soil

In [ ]:
for n,soil in tqdm(enumerate(['Alluvial_Soil/','Black_Soil/','Clay_Soil/','Red_Soil/'])):
    os.chdir(BASE_PATH+'Train/'+soil)
    imgs = os.listdir()

    for image in imgs:
        for i in range(8):
            
            im = Image.open(image)
            
            if im.mode in ("RGBA", "P"):
                im = im.convert("RGB")
            
            if i>0:
                im = augment(im)
            
            path = '/kaggle/working/data/'+soil
            
            filename,extension = image.split('.')
            
            im.save(path+filename+str(i)+'.'+extension)

In [ ]:
if torch.cuda.is_available():
    device = torch.device('cuda')
else:
    device = torch.device('cpu')
    
Model = EfficientNet.from_pretrained('efficientnet-b3', num_classes=4)
Model.to(device)

In [ ]:
LEARNING_RATE = 1e-3
EPOCHS = 100
CLASSES = [0,1,2,3]
EARLY_STOPPING = 25
BATCH_SIZE = 32

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(Model.parameters(),lr=LEARNING_RATE)
scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=5, gamma=0.1)

In [ ]:
os.chdir('/kaggle/working/')

data = datasets.ImageFolder('data/', transform=transform)
generator = DataLoader(data,shuffle=True,batch_size=BATCH_SIZE,pin_memory=True)

In [ ]:
history_accuracy=[]
history_validation_accuracy=[]
history_loss=[]

one_hot = torch.eye(4).to(device)
it = 0
maxi = 0

for epoch in range(1,EPOCHS+1):
    
    running_loss = 0.0 
    correct = 0 
    total = 0
    
    for n,(X,labels) in enumerate(generator):
        X,labels = X.to(device),labels.to(device)
        
        optimizer.zero_grad()
        output = Model(X)
        loss = criterion(output,labels)
        _,predicted = torch.max(output,1)
        
        correct += (predicted == labels).sum().item()
        
        total += labels.size(0)
        accuracy = float(correct) / float(total)

        loss.backward()
        optimizer.step()
        
        running_loss += loss.item()
        
        if n%10==0:
            print( "Epoch : ",epoch," Batch : ", n+1," Loss :  ",running_loss/(n+1)," Accuracy : ",accuracy)
    
    scheduler.step()
            
    history_accuracy.append(correct/total)
    history_loss.append(running_loss/n)
    
    Model.eval()
            
    os.chdir('/kaggle/input/soil-classification-image-data/Soil_Dataset/Test/')
    
    validationtotal = 0
    validationcorrect = 0

    for n,soil in enumerate(['Alluvial_Soil/','Black_Soil/','Clay_Soil/','Red_Soil/']):
        os.chdir(soil)
        imgs = os.listdir()
        
        for image in imgs:
        
            X = transform_test(cv2.imread(image)).unsqueeze_(0).to(device)
            with torch.no_grad():
                output = Model(X).data.cpu().numpy().argmax()
            
            validationtotal += 1
            
            validationcorrect += (output==n)
            
        os.chdir('../')
        
    os.chdir('/kaggle/working/')
    
    if validationcorrect/validationtotal > maxi:
        maxi = validationcorrect/validationtotal
        torch.save(Model,'model '+str(maxi)+'.pth')
        it = 0
    else:
        it += 1
    
    history_validation_accuracy.append(validationcorrect/validationtotal)
    
    if it==EARLY_STOPPING:
        print("Early Stopping....")
        break
    
    print(f"Training Accuracy: {correct/total} Validation Accuracy: {validationcorrect/validationtotal}")
    
    Model.train()

In [ ]:
os.chdir('/kaggle/working/')

In [ ]:
!rm -r data/

In [ ]:
plt.plot(history_validation_accuracy,label='valid')
plt.plot(history_accuracy,label='train')

plt.legend()